In [1]:
import psycopg2
from psycopg2 import Error
import sys
from db_credentials import *
from sql_queries import *
from io import StringIO
import pandas

debug = 0

source_db_config = ''
source_db_name = ''

target_db_config = nba_database_config
target_db_name = nba_database_name

memory_buffer = None
player_data_frame = None

try :
    connection = psycopg2.connect(
        target_db_config
    )

    cursor = connection.cursor()

    print("Successful connection to the database", target_db_name)

except(Exception, Error) as connection_error:
    print("Error while connecting to the database", target_db_name)
    print(connection_error)
    if debug == 1:
        raise
        print("debug!")
    sys.exit()

else :
    try :
        #function you want to run goes here

        memory_buffer = StringIO()

        cursor.copy_expert('''COPY (SELECT a.name, a.draft_team FROM (SELECT name, draft_team, draft_year 
        FROM NBA_LANDING.NBA_PLAYER_DATA WHERE TRIM(draft_year) NOT IN ('1st', '2nd', '8th') ) AS a
        WHERE cast(a.draft_year as integer) > 2003) 
        TO STDIN WITH CSV HEADER DELIMITER '|';''', memory_buffer)

        memory_buffer.seek(0)

        player_data_frame = pandas.read_csv(memory_buffer, sep='|')
        
        print(player_data_frame.to_string())
        
    except (Exception, Error) as sql_error:
        print(sql_error)
        if connection:
            connection.close()
            print("Successful disconnection from the database", target_db_name)
            sys.exit()

    if connection:
            connection.close()
            print("Successful disconnection from the database", target_db_name)

Successful connection to the database nba
                         name                         draft_team
0                Alex Abrines              Oklahoma City Thunder
1                  Alex Acker                    Detroit Pistons
2                  Quincy Acy                    Toronto Raptors
3                Hassan Adams                    New Jersey Nets
4                Jordan Adams                  Memphis Grizzlies
5                Steven Adams              Oklahoma City Thunder
6                 Bam Adebayo                         Miami Heat
7               Arron Afflalo                    Detroit Pistons
8                Maurice Ager                   Dallas Mavericks
9               Alexis Ajinca                  Charlotte Bobcats
10              Solomon Alabi                   Dallas Mavericks
11             Furkan Aldemir               Los Angeles Clippers
12               Cole Aldrich                New Orleans Hornets
13          LaMarcus Aldridge                   

In [7]:
player_data_frame.pivot('draft_team','name')

C:\Users\golfn\AppData\Local\Temp\ipykernel_17304\1601476977.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  player_data_frame.pivot('draft_team','name')


draft_team
Atlanta Hawks
Boston Celtics
Brooklyn Nets
Charlotte Bobcats
Charlotte Hornets
Chicago Bulls
Cleveland Cavaliers
Dallas Mavericks
Denver Nuggets
Detroit Pistons
